# Analysis of SI Table 6 for genes overexpressed in cancer

**Genes overexpressed in different human solid cancers exhibit different tissue-specific expression profiles**

In [1]:
%load_ext blackcellmagic
from glob import glob as glob
from io import StringIO
import os as os

import pandas as pd
import urllib
import json

To make things easier, I'm going to split Table 6 into separate tables for each type of cancer and then save as a `csv`.

In [2]:
file_list = glob("*.csv")

In [3]:
file_list = [i for i in file_list if "ligands" not in i]

In [4]:
file_list

['melanoma.csv',
 'endometrial.csv',
 'lung.csv',
 'kidney.csv',
 'colon.csv',
 'ovarian.csv',
 'astrocytoma.csv',
 'liver.csv',
 'thyroid.csv',
 'breast.csv',
 'glioblastoma.csv',
 'prostate.csv']

Now, define the helper functions.

In [5]:
def get_uniprot(gene):
    """
    Look up gene name in Uniprot.
    """
    # print(f"Looking up {gene} in Uniprot database...")
    url = f"https://www.uniprot.org/uniprot/?query=reviewed:yes+AND+organism:9606+AND+gene_exact:{gene}&format=tab"
    request = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        # print(f"{gene:10}: HTTP error. Skipping...")
        return None
    page = response.read()
    if page == b"":
        print(f"{gene:10}: Response error. Skipping...")
        return None
    else:
        return pd.read_csv(StringIO(page.decode("utf-8")), sep="\t")


In [6]:
def get_bindingdb(uniprot):
    """
    Look up uniprot in Binding DB.
    """
    # print(f"Looking up {uniprot} in BindingDB...")
    url = f"http://www.bindingdb.org/axis2/services/BDBService/getLigandsByUniprots?uniprot={uniprot}&cutoff=10&code=0&response=application/json"
    request = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        # print(f"{uniprot:10}: HTTP error. Skipping...")
        return None

    page = response.read()
    binding_db = json.loads(page.decode("utf-8"))
    if binding_db["getLigandsByUniprotsResponse"] == "":
        return None
    return pd.DataFrame(binding_db["getLigandsByUniprotsResponse"]["affinities"])


In [7]:
def bindingdb_to_table(table):
    """
    Parse the Binding DB `json` response.
    """

    query = table["query"].values[0]
    affinities = table["affinity"].values
    affinity_types = table["affinity_type"].values
    smiles = table["smile"].values
    monomers = table["monomerid"].values
    return pd.DataFrame(
        {
            "Affinities": affinities,
            "Affinity type": affinity_types,
            "SMILES": smiles,
            "MonomerID": monomers,
            "Query": query,
        }
    )


In [8]:
def manual_pivot(table):
    """
    The data we get from Binding DB will be in rows.
    But we want each row to basically be a new column, so that gene names are the rows.
    """
    # I had problems working with the built-in `pivot` because the data is non-numeric
    # tmp3 = tmp2.pivot_table(index="Query",
    #                        columns=tmp2.index,
    #                        values=["Affinities"],
    #                        aggfunc=lambda x: ' '.join(x))

    df = pd.DataFrame()
    for row in range(len(table)):
        df[f"Affinity {row:03d}"] = pd.Series(table["Affinities"][row])
        df[f"Type {row:03d}"] = pd.Series(table["Affinity type"][row])
        df[f"SMILES {row:03d}"] = pd.Series(table["SMILES"][row])
        df[f"ID {row:03d}"] = pd.Series(table["MonomerID"][row])
    return df


In [ ]:
def filter_top_three_ligands(table):
    """
    Extract the top three ligands with the tightest binding.
    `errors="coerce"` will ignore entries like >10 or <10.
    """
    table["Affinities"] = pd.to_numeric(table["Affinities"], errors="coerce")
    return table.nsmallest(3, "Affinities")

Loop over the files, then loop over the genes.

To start I'm just looping over the breast cancer data to see if I can match the one file that Tiqing returned.

First, I map from gene to protein and Uniprot ID. That is stored in the `gene_df` DataFrame.
Then I use the Uniprot ID to look up data from Binding DB.

Then I join `gene_df` and the Binding DB data.

In [ ]:
for file in file_list:
    if os.path.isfile(os.path.splitext(file)[0] + "-ligands.csv"):
        print(f"Found {os.path.splitext(file)[0] + '-ligands.csv'}, skipping...")
        continue
        
    table = pd.read_csv(file, skiprows=1, names=["Unknown", "Gene", "Overexpression"])
    print(f"Loaded {len(table)} genes from {file}...")
    table = table.drop_duplicates(subset="Gene", keep="first")
    print(f"Found {len(table)} unique genes...")
    df = pd.DataFrame()
    for gene, overexpression in zip(
        table["Gene"].values, table["Overexpression"].values
    ):

        if gene == "---":
            continue
        if "///" in gene:
            gene = gene.split("///")[0]

        gene_table = get_uniprot(gene)
        if gene_table is None:
            print(f"{gene:10} → Unknown. Skipping...")
            continue
        uniprot = gene_table["Entry"].values[0]
        protein = gene_table["Protein names"].values[0]

        gene_df = pd.DataFrame()
        gene_df["Gene"] = pd.Series(gene)
        gene_df["Uniprot"] = pd.Series(uniprot)
        gene_df["Protein"] = pd.Series(protein)
        gene_df["Overexpression"] = pd.Series(overexpression)

        binding_db = get_bindingdb(uniprot)

        if binding_db is not None:
            print(f"{gene:10} → {uniprot:10} → {len(binding_db):4} ligands found...")
            binding_table = bindingdb_to_table(binding_db)
            trimmed_table = filter_top_three_ligands(binding_table)
            binding_pivot = manual_pivot(trimmed_table)

            gene_df = gene_df.join(binding_pivot)
            # Only track the proteins with entries in BindingDB
            df = df.append(gene_df, ignore_index=True)
        else:
            print(f"{gene:10} → {uniprot:10} → {'0':>4} ligands found...")
            pass
    
    if df.empty:
        continue
        
    column_list = df.columns.tolist()
    column_end = [i for i in column_list if "Affinity" in i]
    column_max = int(column_end[-1].split(" ")[1])
    column_order = ["Gene", "Uniprot", "Protein", "Overexpression"]
    for i in range(column_max + 1):
        column_order.append(f"Affinity {i:03d}")
        column_order.append(f"Type {i:03d}")
        column_order.append(f"SMILES {i:03d}")
        column_order.append(f"ID {i:03d}")
    df = df[column_order]
    df.to_csv(os.path.splitext(file)[0] + "-ligands.csv")


Found melanoma-ligands.csv, skipping...
Found endometrial-ligands.csv, skipping...
Found lung-ligands.csv, skipping...
Found kidney-ligands.csv, skipping...
Loaded 89 genes from colon.csv...
Found 73 unique genes...
COL11A1    → P12107     →    0 ligands found...
KLK10      → O43240     →    0 ligands found...
CST1       → P01037     →    0 ligands found...
IL8        → P10145     →    0 ligands found...
COL10A1    → Q03692     →    0 ligands found...
INHBA      → P08476     →    0 ligands found...
CXCL5      → P42830     →    0 ligands found...
MMP1       → P03956     →    0 ligands found...
CDH3       → P55291     →    0 ligands found...
TACSTD2    → P09758     →    0 ligands found...
CHI3L1     → P36222     →    0 ligands found...
SPP1       → P10451     →    0 ligands found...
MMP7       → P09237     →    0 ligands found...
KRT23      → Q9C075     →    0 ligands found...
FAP        → Q12884     →    0 ligands found...
MSLN       → Q13421     →    0 ligands found...
CXCL3      → P19

KeyboardInterrupt: 

> /home/dslochower/data/applications/anaconda3/lib/python3.6/ssl.py(689)do_handshake()
    687     def do_handshake(self):
    688         """Start the SSL/TLS handshake."""
--> 689         self._sslobj.do_handshake()
    690         if self.context.check_hostname:
    691             if not self.server_hostname:

